Kawasaki Quantum Summer Camp 2025

# 量子機械学習


Kifumi Numata, IBM Quantum (Jul 31, 2025)

In [ ]:
import numpy as np

# 描画のため
import matplotlib.pyplot as plt

# Scikit-learnのインポート(Python用の機械学習ライブラリー)
from sklearn import datasets
from sklearn.model_selection import train_test_split # データ分割
from sklearn.svm import SVC # SVM Classification(SVM分類)
from sklearn.decomposition import PCA # Principal component analysis(主成分分析)
from sklearn.preprocessing import StandardScaler, MinMaxScaler # 標準化、正規化のスケール変換

# Qiskitのインポート
from qiskit_aer import AerSimulator
from qiskit.circuit import QuantumCircuit, Parameter
from qiskit.circuit.library import zz_feature_map
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit.circuit.library import unitary_overlap
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

## データを用意

ここでは，手書き数字画像のデータセット(MNISTデータセット)から0と1のサブセットを扱います。

In [ ]:
# 数字データセットから2クラスのデータ(0と1)を読み込み
digits = datasets.load_digits(n_class=2)   

# 読み込んだ画像の最初の100枚をプロット
fig, axes = plt.subplots(10, 10, figsize=(15, 15), subplot_kw={'xticks':[], 'yticks':[]}, gridspec_kw=dict(hspace=0.5, wspace=0.5))
for i, ax in enumerate(axes.flat):
    ax.set_axis_off()
    ax.imshow(digits.images[i], cmap=plt.cm.gray_r, interpolation='nearest')
    ax.set_title(digits.target[i])

In [ ]:
print(digits.images[0])

このデータセットには、合計360個のデータが含まれています。各データポイントは、8×8の数字の画像で、配列になっていて、各要素は0（白）から16（黒）までの整数です。古典的な分類アルゴリズムの際と同様に、データセットを学習用（25個）とテスト用（10個）のサンプルに分割し、正規化する必要があります。このデータセットを量子分類アルゴリズムに用いるために、範囲を-1から1の間にスケーリングし、次元を使用する量子ビット数（今回は4）に縮小します。

In [ ]:
# データセットの分割
sample_train, sample_test, labels_train, labels_test = train_test_split(
     digits.data, digits.target, test_size=0.4, random_state=22)

# 次元削除
n_dim = 4
pca = PCA(n_components=n_dim).fit(sample_train)
sample_train = pca.transform(sample_train)
sample_test = pca.transform(sample_test)

# 正規化
std_scale = StandardScaler().fit(sample_train)
sample_train = std_scale.transform(sample_train)
sample_test = std_scale.transform(sample_test)

# スケーリング
samples = np.append(sample_train, sample_test, axis=0)
minmax_scale = MinMaxScaler((-1, 1)).fit(samples)
sample_train = minmax_scale.transform(sample_train)
sample_test = minmax_scale.transform(sample_test)

# 学習用25個とテスト用10個を選択
train_size = 25
sample_train = sample_train[:train_size]
labels_train = labels_train[:train_size]

test_size = 10
sample_test = sample_test[:test_size]
labels_test = labels_test[:test_size]

In [ ]:
# 一つ目のデータをそれぞれ表示
print(sample_train[0], labels_train[0])
print(sample_test[0], labels_test[0])

In [ ]:
# 学習用データのラベルを表示
print(labels_train)

## データ符号化(エンコード)

この古典データを、量子特徴量マップ(`zz_feature_map`)を用いて量子状態空間にエンコードしていきます。

In [ ]:
# 4特徴量、深さ(繰り返し数)1のZZFeatureMap
zz_map = zz_feature_map(feature_dimension=4, entanglement='linear', reps=1, insert_barriers=True)
zz_map.draw('mpl')

### 量子カーネルの計算

`zz_feature_map`を使って、学習データの0個目と1個目のデータについて、量子カーネルを計算する量子回路を作成し、実際に計算してみます。この値は、二つの量子状態のFidelity(忠実度)ともいいます。

In [ ]:
print(sample_train[0])
print(sample_train[1])

In [ ]:
zz_map = zz_feature_map(feature_dimension=4, reps=1, entanglement='linear')
qc_1 =zz_map.assign_parameters(sample_train[0])
qc_2 =zz_map.assign_parameters(sample_train[1])
fidelity_circuit = qc_1.copy()
fidelity_circuit.append(qc_2.inverse().decompose(), range(fidelity_circuit.num_qubits))
fidelity_circuit.measure_all()
fidelity_circuit.decompose().draw('mpl')

各回転ゲートのパラメーター値は少し読みにくいですが、回路が対称になっていることが分かると思います。左半分は学習データsample_train[0]が、右半分は学習データsample_train[1]がコード化されています。

例として、上記の量子カーネルを測定し、ゼロ状態のカウント数の割合として、カーネル行列の要素（訓練データの0個目と1個目のデータについて）を計算します。

In [ ]:
# シミュレーターで実験
backend = AerSimulator()
# 回路を最適化
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_qc = pm.run(fidelity_circuit)

In [ ]:
# Samplerで実行
num_shots = 4096
sampler = Sampler(backend)
job = sampler.run([isa_qc], shots=num_shots)
result = job.result()

#  測定された回数を表示
counts = result[0].data.meas.get_counts()
print(counts)

In [ ]:
counts.get('0000')/nshots

### 量子カーネル計算

このプロセスを，学習データサンプルのペアごとに繰り返して学習カーネル行列を埋めていきます。
先ほどど同じ `zz_feature_map` を用いて量子回路を作り、また、今回は、 `unitary_overlap`を使ってカーネル行列の要素となる内積(fidelity)を計算します。

In [ ]:
zz_map = zz_feature_map(feature_dimension=4, reps=1, entanglement='linear')
fidelity = unitary_overlap(zz_map, zz_map)
fidelity.measure_all()

回路の形状を見てみましょう。

In [ ]:
fidelity.draw('mpl', idle_wires=False, fold=-1)

出力を格納するために、空のカーネル行列と、空のテスト用カーネル行列を作ります。

In [ ]:
# 空のカーネル行列
kernel_matrix = np.full((train_size, train_size), np.nan) # 25x25の学習用カーネル行列
test_matrix = np.full((test_size, train_size), np.nan) # 10x25のテスト用カーネル行列

Overlap回路を作ります。カーネル行列は対角に対して対称なので、同じ計算は繰り返さないようにすることで、計算回数は二分の一に減らせます。また、ノイズがない場合は対角要素が1となるため、今回は対角要素を計算せず、1を代入します。(25×25×1/2-25)要素と(10×25)要素を計算するので、以下のセルは実行にすこし時間がかかるかもしれません。

In [ ]:
%%time
num_shots = 2048

backend = AerSimulator()
sampler = Sampler(mode=backend)

for x1 in range(0,train_size):
    for x2 in range(x1+1,train_size):
        unitary1 = zz_map.assign_parameters(list(sample_train[x1]))
        unitary2 = zz_map.assign_parameters(list(sample_train[x2]))

        # Overlap回路の作成
        overlap_circ = unitary_overlap(unitary1, unitary2)
        overlap_circ.measure_all()
    
        # Qiskit sampler primitiveの実行
        counts = (
            sampler.run([overlap_circ], shots=num_shots).result()[0].data.meas.get_int_counts()
        )

        # |0>状態の確率をカーネル行列の要素に入れ、転置された要素にも同じ確率を入れます。
        kernel_matrix[x1, x2] = counts.get(0, 0.0)/num_shots
        kernel_matrix[x2, x1] = counts.get(0, 0.0)/num_shots
    # 確率の内積なので、対角線上の要素を1で埋めます
    kernel_matrix[x1,x1] =1    

print("training done")

# 上記と同様のプロセスをテストデータを使って行う
for x1 in range(0,test_size):
    for x2 in range(0,train_size):
        unitary1 = zz_map.assign_parameters(list(sample_test[x1]))
        unitary2 = zz_map.assign_parameters(list(sample_train[x2]))

        # Overlap回路の作成
        overlap_circ = unitary_overlap(unitary1, unitary2)
        overlap_circ.measure_all()
    
        counts = (
            sampler.run([overlap_circ], shots=num_shots).result()[0].data.meas.get_int_counts()
        )

        test_matrix[x1, x2] = counts.get(0, 0.0)/num_shots

print("test matrix done")

2つのカーネル行列を見てみましょう。

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].imshow(np.asmatrix(kernel_matrix),
              interpolation='nearest', origin='upper', cmap='Blues')
axs[0].set_title("kernel matrix")
axs[1].imshow(np.asmatrix(test_matrix),
              interpolation='nearest', origin='upper', cmap='Reds')
axs[1].set_title("test matrix")
plt.show()

### 古典計算による学習

量子カーネル法からカーネル行列と同様の形式の test_matrix が得られたので、古典の機械学習アルゴリズムを適用してテストデータの予測を行い、その精度をチェックすることができます。Scikit-Learnのサポートベクトル分類器（SVC）`sklearn.svc`を使い、事前に計算したカーネルを使うため、`kernel = precomputed` を指定します。

In [ ]:
# 古典の機械学習パッケージからサポートベクトル分類器で事前に計算されたカーネル行列を使用することを指定
qml_svc = SVC(kernel="precomputed")

`SVC.fit` を使って、カーネル行列と学習用ラベルを入力し、学習します。`SVC.predict` は test_matrix を使ってテストデータのラベルを予測します。`SVC.score` はテストデータをスコアリングして精度を返します。

In [ ]:
# 事前に計算された行列と学習データのラベルを入力。古典アルゴリズムがfitを行います。
qml_svc.fit(kernel_matrix, labels_train)

# 学習データとテストデータのカーネル行列を用いてテストデータのラベルを予測します。
label_predict = qml_svc.predict(test_matrix)
print(label_predict, labels_test, sep="\n")

# テストデータの行列とテストラベルを入力として、.scoreを使ってデータをテストします。
qml_score_precomputed_kernel = qml_svc.score(test_matrix, labels_test)
print(f"Precomputed kernel classification test score: {qml_score_precomputed_kernel}")

90%の確率で学習されました。

# 洋服画像データの分類


ここで扱うデータは，MNISTデータセットの亜種である[Fashion-MNIST](https://github.com/zalandoresearch/fashion-mnist/blob/master/README.ja.md)という洋服画像データセットのサブセットです。


![Fashion-MNIST](https://raw.githubusercontent.com/zalandoresearch/fashion-mnist/refs/heads/master/doc/img/fashion-mnist-sprite.png)


以下のラベルの画像について分類します。

- label 2: プルオーバー
- label 3: ドレス

まずデータセットを読み込んで，クラスごとに1枚ずつ画像を表示してみます。

In [ ]:
# データのロード
DATA_PATH = 'fashion.npz'
data = np.load(DATA_PATH)

sample_train = data['sample_train']
labels_train = data['labels_train']
sample_test = data['sample_test']

# データセットの分割
sample_train, sample_test, labels_train, labels_test = train_test_split(
    sample_train, labels_train, test_size=0.2, random_state=42)

# データの表示
fig = plt.figure()
LABELS = [2,3]
num_labels = len(LABELS)
for i in range(num_labels):
    ax = fig.add_subplot(2, num_labels, i+1)
    img = sample_train[labels_train==LABELS[i]][0].reshape((28, 28))
    ax.imshow(img, cmap="Greys")

次に，以下のデータセットの前処理をします

- 主成分分析(PCA)による次元圧縮
- 正規化
- スケーリング
- 学習用（80個）とテスト用（10個）のサンプルを選択


In [ ]:
# 次元削除
N_DIM = 4
pca = PCA(n_components=N_DIM).fit(sample_train)
sample_train = pca.transform(sample_train)
sample_test = pca.transform(sample_test)

# 正規化
std_scale = StandardScaler().fit(sample_train)
sample_train = std_scale.transform(sample_train)
sample_test = std_scale.transform(sample_test)

# スケーリング
samples = np.append(sample_train, sample_test, axis=0)
minmax_scale = MinMaxScaler((-1, 1)).fit(samples)
sample_train = minmax_scale.transform(sample_train)
sample_test = minmax_scale.transform(sample_test)

# 選択
train_size = 80
sample_train = sample_train[:train_size]
labels_train = labels_train[:train_size]

test_size = 10
sample_test = sample_test[:test_size]
labels_test = labels_test[:test_size]

In [ ]:
# 一つ目のデータをそれぞれ表示
print(sample_train[0], labels_train[0])
print(sample_test[0], labels_test[0])

## 演習

この洋服画像データについて、特徴量マップ(`zz_feature_map`)を使ってカーネル行列を計算し、古典のSVMを使って学習してみます。学習率を確認してみましょう。「#コードを入れてください」にコードを記入して、コードを完成させてください。


In [ ]:
zz_map =  #コードを入れてください -1
fidelity = #コードを入れてください -2
fidelity.measure_all()
fidelity.decompose().draw('mpl', idle_wires=False, fold=-1)

In [ ]:
# 空のカーネル行列
kernel_matrix = np.full((train_size, train_size), np.nan) # 80x80の学習用カーネル行列
test_matrix = np.full((test_size, train_size), np.nan) # 10x80のテスト用カーネル行列

カーネル行列を計算します。

In [ ]:
%%time
num_shots = 1024

backend = AerSimulator()
sampler = Sampler(mode=backend)

for x1 in range(0,train_size):
    for x2 in range(x1+1,train_size):
        unitary1 = #コードを入れてください -3
        unitary2 = #コードを入れてください -4

        # Overlap回路の作成
        overlap_circ = unitary_overlap(unitary1, unitary2)
        overlap_circ.measure_all()
    
        # Qiskit sampler primitiveの実行
        counts = (
            sampler.run([overlap_circ], shots=num_shots).result()[0].data.meas.get_int_counts()
        )

        # |0>状態の確率をカーネル行列の要素に入れ、転置された要素にも同じ確率を入れます。
        kernel_matrix[x1, x2] = counts.get(0, 0.0)/num_shots
        kernel_matrix[x2, x1] = counts.get(0, 0.0)/num_shots
    # 確率の内積なので、対角線上の要素を1で埋めます
    kernel_matrix[x1,x1] =1    

print("training done")

# 上記と同様のプロセスをテストデータを使って行う
for x1 in range(0,test_size):
    for x2 in range(0,train_size):
        unitary1 = #コードを入れてください -5
        unitary2 = #コードを入れてください -6

        # Overlap回路の作成
        overlap_circ = unitary_overlap(unitary1, unitary2)
        overlap_circ.measure_all()
    
        counts = (
            sampler.run([overlap_circ], shots=num_shots).result()[0].data.meas.get_int_counts()
        )

        test_matrix[x1, x2] = counts.get(0, 0.0)/num_shots

print("test matrix done")

行列を表示します。

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].imshow(np.asmatrix(kernel_matrix),
              interpolation='nearest', origin='upper', cmap='Blues')
axs[0].set_title("kernel matrix")
axs[1].imshow(np.asmatrix(test_matrix),
              interpolation='nearest', origin='upper', cmap='Reds')
axs[1].set_title("test matrix")
plt.show()

計算された量子カーネル行列を使って古典計算で学習します。

In [ ]:
# 古典の機械学習パッケージからサポートベクトル分類器で事前に計算されたカーネル行列を使用することを指定
qml_svc = SVC(kernel="precomputed")

# 事前に計算された行列と学習データのラベルを入力。古典アルゴリズムがfitを行います。
qml_svc.fit(kernel_matrix, labels_train)

# 学習データとテストデータのカーネル行列を用いてテストデータのラベルを予測します。
label_predict = qml_svc.predict(test_matrix)
print(label_predict, labels_test, sep="\n")

# テストデータの行列とテストラベルを入力として、.scoreを使ってデータをテストします。
qml_score_precomputed_kernel = qml_svc.score(test_matrix, labels_test)
print(f"Precomputed kernel classification test score: {qml_score_precomputed_kernel}")

## 時間の余った方向け

データセットunknown_dataが プルオーバー（ラベル2）、またはドレス（ラベル3）、どちらのデータセットであるかを学習した量子カーネル行列を使って、SVMで判別してください。データunknown_dataは、10個の同じラベルのデータセットを次元削除、正規化、スケーリングしたものです。

In [ ]:
unknown_data =[[-0.13116065, -0.06375504, -0.13685728,  0.84755115],
                [-0.37941815, -0.76067967, -0.22019793,  0.17362356],
               [-0.32055824, -0.32319408, -0.36608317,  0.68697767],
               [ 0.31559114, -0.22212598, -0.01367073,  0.26984296],
               [ 0.61950235,  0.11437916, -0.71187256, -0.20481759],
               [ 0.81223033,  0.16992666,  0.0184564,   0.03393165],
               [ 0.84295123,  0.07871908, -0.73484068,  0.05869072],
               [-0.32055824, -0.32319408, -0.36608317,  0.68697767],
               [ 0.31559114, -0.22212598, -0.01367073,  0.26984296],
               [ 0.61950235,  0.11437916, -0.71187256, -0.20481759]]

In [ ]:
%%time
# 学習データとunknown_dataデータとのカーネル行列(80x10)を計算します











print("test matrix done")

In [ ]:
# 学習データとunknown_dataデータのカーネル行列を用いてテストデータのラベルを予測します。
label_predict = qml_svc.predict(test_matrix)
print(label_predict)

In [ ]:
import qiskit
qiskit.__version__